## 전리뷰 크롤링

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import pymysql
import sqlite3
import csv

# 버전 충돌
import collections
collections.Callable = collections.abc.Callable


In [3]:
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

url = 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000161508&t_page=%ED%86%B5%ED%95%A9%EA%B2%80%EC%83%\
    89%EA%B2%B0%EA%B3%BC%ED%8E%98%EC%9D%B4%EC%A7%80&t_click=%EA%B2%80%EC%83%89%EC%83%81%ED%92%88%EC%83%81%EC%84%B8&t_search_name=\
        %EB%8F%85%EB%8F%84%20%EB%A1%9C%EC%85%98&t_number=1&dispCatNo=1000001000100160001&trackingCd=Result_1'

driver.get(url)

click_reviews= driver.find_element(By.CLASS_NAME, "goods_reputation")
click_reviews.click()
time.sleep(2)

id=[] # 작성ID
star_rate=[] # 별점
review=[] # 리뷰 내용

## 1 페이지
page_html = driver.page_source
soup = BeautifulSoup(page_html,'html.parser')

# 리뷰 리스트
reviews_lst = soup.select('div.review_list_wrap > ul.inner_list > li')

for idx in range(len(reviews_lst)):
    # 작성자
    id.append(reviews_lst[idx].select('a.id')[0].text)
    # 별점
    star_rate.append(reviews_lst[idx].select('span.review_point')[0].text)
    # 내용
    review.append(reviews_lst[idx].select('div.txt_inner')[0].text)

print(id)
##################################### 2페이지부터 나머지~~#########################################################
page_num=1
count=1

while True:
    try:

        link = driver.find_element(By.XPATH,f'//*[@id="gdasContentsArea"]/div/div[8]/a[{page_num}]')
        link.click()
        time.sleep(3)

        page_html = driver.page_source
        soup = BeautifulSoup(page_html,'html.parser')

        # 리뷰 리스트
        reviews_lst = soup.select('div.review_list_wrap > ul.inner_list > li')
        print(len(reviews_lst))

        for idx in range(len(reviews_lst)):
            # 작성자
            id.append(reviews_lst[idx].select('a.id')[0].text)
            # 별점
            star_rate.append(reviews_lst[idx].select('span.review_point')[0].text)
            # 내용
            review.append(reviews_lst[idx].select('div.txt_inner')[0].text)

        # 페이지
        if count>=10:
            page_num = 2
            count=1
        else:
            count+=1
            page_num += 1

    except NoSuchElementException:
        print(f'{page_num}페이지를 찾을수 없습니다. 크롤링 끝-')
        break



['teatime', '러블리윤', '올영여행', '네니넨', '멍창이', '맑고밝게', '본예이', '올영세상', '뜬뜬뜬', '러블리윤']
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
11페이지를 찾을수 없습니다. 크롤링 끝-


In [5]:
print(len(id),len(star_rate),len(review))

1000 1000 1000


In [6]:
review_df = pd.DataFrame({'id':id,'star_rate':star_rate, 'review':review })

In [7]:
review_df.head()

,id,star_rate,review
0,teatime,5점만점에 5점,가성비 너무 좋은 세트예요!제 첫 라운드랩 로션인데 아주 만족해요.로션이 물듯이 가...
1,러블리윤,5점만점에 3점,중1 남아가 사용중인데토너와 달리 로션은 유분기가 느껴지네요유분기로 인해 트러블 발...
2,올영여행,5점만점에 5점,"라운드랩제품 좋아하고, 독도 제품 순해서 로션,토너 모두 좋아요."
3,네니넨,5점만점에 5점,독도로션 리뷰입니다.로션을 찾다가 세일하니까 요 제품이 용량대비 엄청 저렴한 편이더...
4,멍창이,5점만점에 5점,원래 크림만 사용하다가 피부과에서 로션도 필수라길래 구매했어요. 확실히 너무 좋아요...


In [8]:
# 별점 숫자로 변환
review_df['star_rate'] = review_df['star_rate'].apply(lambda x: int(x.split(' ')[-1].replace('점','')))

In [9]:
review_df['brand'] = '라운드랩'
review_df['prod_name'] = '라운드랩 1025 독도 로션'
review_df = review_df[['brand','prod_name','id','star_rate','review']]
review_df.head()

,brand,prod_name,id,star_rate,review
0,라운드랩,라운드랩 1025 독도 로션,teatime,5,가성비 너무 좋은 세트예요!제 첫 라운드랩 로션인데 아주 만족해요.로션이 물듯이 가...
1,라운드랩,라운드랩 1025 독도 로션,러블리윤,3,중1 남아가 사용중인데토너와 달리 로션은 유분기가 느껴지네요유분기로 인해 트러블 발...
2,라운드랩,라운드랩 1025 독도 로션,올영여행,5,"라운드랩제품 좋아하고, 독도 제품 순해서 로션,토너 모두 좋아요."
3,라운드랩,라운드랩 1025 독도 로션,네니넨,5,독도로션 리뷰입니다.로션을 찾다가 세일하니까 요 제품이 용량대비 엄청 저렴한 편이더...
4,라운드랩,라운드랩 1025 독도 로션,멍창이,5,원래 크림만 사용하다가 피부과에서 로션도 필수라길래 구매했어요. 확실히 너무 좋아요...


In [10]:
import pandas as pd
import re


# 이모티콘 제거 함수 정의
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # 이모지
                           u"\U0001F300-\U0001F5FF"  # 이모티콘 & 기호
                           u"\U0001F680-\U0001F6FF"  # 트랜스포터 & 심볼
                           u"\U0001F700-\U0001F77F"  # 임베드 & 심볼 보충
                           u"\U0001F780-\U0001F7FF"  # 야구 보충
                           u"\U0001F800-\U0001F8FF"  # 아케이드 보충
                           u"\U0001F900-\U0001F9FF"  # 장난감 보충
                           u"\U0001FA00-\U0001FA6F"  # 음식 보충
                           u"\U0001FA70-\U0001FAFF"  # 공항 보충
                           u"\U0001FB00-\U0001FBFF"  # 추가 일본 보충
                           u"\U0001FC00-\U0001FCFF"  # 추가 한국 보충
                           u"\U0001F004-\U0001F0CF"  # 추가 스포츠 보충
                           u"\U0001F170-\U0001F251"
                           u"\U0001F004-\U0001F0CF"
                           u"\U0001F300-\U0001F5FF"
                           u"\U0001F004-\U0001F0CF"
                           u"\U0001F600-\U0001F64F"
                           u"\U0001F680-\U0001F6FF"
                           u"\U0001F700-\U0001F77F"
                           u"\U0001F780-\U0001F7FF"
                           u"\U0001F800-\U0001F8FF"
                           u"\U0001F900-\U0001F9FF"
                           u"\U0001FA00-\U0001FA6F"
                           u"\U0001FA70-\U0001FAFF"
                           u"\U0001FB00-\U0001FBFF"
                           u"\U0001FC00-\U0001FCFF"
                           u"\U0001F004-\U0001F0CF"
                           u"\U0001F170-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# 'review' 열에서 이모티콘 제거
review_df['review'] = review_df['review'].apply(remove_emoji)

# 결과 데이터프레임 확인
print(review_df)


    brand        prod_name           id  star_rate  \
0    라운드랩  라운드랩 1025 독도 로션      teatime          5   
1    라운드랩  라운드랩 1025 독도 로션         러블리윤          3   
2    라운드랩  라운드랩 1025 독도 로션         올영여행          5   
3    라운드랩  라운드랩 1025 독도 로션          네니넨          5   
4    라운드랩  라운드랩 1025 독도 로션          멍창이          5   
..    ...              ...          ...        ...   
995  라운드랩  라운드랩 1025 독도 로션  leeyj01****          5   
996  라운드랩  라운드랩 1025 독도 로션      sen****          5   
997  라운드랩  라운드랩 1025 독도 로션         꾸꾸망개          5   
998  라운드랩  라운드랩 1025 독도 로션         꾸꾸망개          5   
999  라운드랩  라운드랩 1025 독도 로션    prototype          5   

                                                review  
0    가성비 너무 좋은 세트예요!제 첫 라운드랩 로션인데 아주 만족해요.로션이 물듯이 가...  
1    중1 남아가 사용중인데토너와 달리 로션은 유분기가 느껴지네요유분기로 인해 트러블 발...  
2                 라운드랩제품 좋아하고, 독도 제품 순해서 로션,토너 모두 좋아요.  
3    독도로션 리뷰입니다.로션을 찾다가 세일하니까 요 제품이 용량대비 엄청 저렴한 편이더...  
4    원래 크림만 사용하다가 피부과에서 로션도 필수라길래 구매했어요. 확실히 너무 좋아요...  
..       

In [11]:
review_df['review']

0      가성비 너무 좋은 세트예요!제 첫 라운드랩 로션인데 아주 만족해요.로션이 물듯이 가...
1      중1 남아가 사용중인데토너와 달리 로션은 유분기가 느껴지네요유분기로 인해 트러블 발...
2                   라운드랩제품 좋아하고, 독도 제품 순해서 로션,토너 모두 좋아요.
3      독도로션 리뷰입니다.로션을 찾다가 세일하니까 요 제품이 용량대비 엄청 저렴한 편이더...
4      원래 크림만 사용하다가 피부과에서 로션도 필수라길래 구매했어요. 확실히 너무 좋아요...
                             ...                        
995    무난하게 남녀노소 사용하기 좋아요! 복합성 피부가 사용해도 좋아요 너무 기름지지도 ...
996    제형이 묽어서 가볍게 발려요. 사계절 다 쓰기 좋을 것 같아요.  보습효과도 괜찮아...
997    피부가 극건성에 아토피에 엄청 예민한 피부인 친언니가 잘써서 저도 따라 쓰다가 아예...
998    스킨 케어 단계에서 유분기가 도는 걸 사용하게 되면 다음 날에 트러블이 올라오게 되...
999    *제형*완전한 물제형입니다. *추천 피부타입*솔직히 모든 피부타입에게 다 맞을 것 ...
Name: review, Length: 1000, dtype: object

In [12]:
review_df.to_csv('./data/02.데이터수집_01크롤링_올리브영_리뷰_dokdo_review.csv', index=False, encoding='utf-8-sig')

### DB에 밀어넣기

In [13]:
# csv_file ='./data/02.데이터수집_01크롤링_올리브영_리뷰_dokdo_review.csv'
# df = pd.read_csv(csv_file)
# df

In [14]:
# conn = pymysql.connect(host='localhost', user='root', password='1111',\
#     db = 'beaulol', charset='utf8')
# cur= conn.cursor()

# f=open('./data/02.데이터수집_01크롤링_올리브영_리뷰_dokdo_review.csv','r')

# csvReader = list(csv.reader(f))
# cur.execute("create table if not exists review_db ( brand VARCHAR(20), prod_name VARCHAR(40), id VARCHAR(20),\
# star_rate INT, review TEXT, PRIMARY KEY (id))")


# for data in csvReader[1:]:
#     row1= data[0]
#     row2= data[1]
#     row3= data[2]
#     row4= int(data[3])
#     row5= data[4]

#     print(row3,row4,type(row4))

#     try:
#         sql = """insert into review_db (brand, prod_name, id, star_rate, review) values(%s, %s, %s, %s, %s);"""
#         cur.execute(sql,(row1,row2,row3,row4,row5))
#     except:
#         continue

# f.close()
# conn.commit()


# conn.close()